# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 1:14PM,257269,10,MSP218439,"Methapharm, Inc.",Released
1,Feb 13 2023 1:14PM,257269,10,MSP218490,"Methapharm, Inc.",Released
2,Feb 13 2023 1:14PM,257269,10,MSP218495,"Methapharm, Inc.",Released
3,Feb 13 2023 1:04PM,257268,10,SOIVA0000583,Ivaoes Animal Health,Released
4,Feb 13 2023 12:38PM,257265,10,PRF-52703,Bio-PRF,Released
5,Feb 13 2023 12:38PM,257265,10,PRF-52707,Bio-PRF,Released
6,Feb 13 2023 12:38PM,257265,10,PRF-52708,Bio-PRF,Released
7,Feb 13 2023 12:38PM,257265,10,PRF-52713,Bio-PRF,Released
8,Feb 13 2023 12:38PM,257265,10,9153370,Bio-PRF,Released
9,Feb 13 2023 12:38PM,257265,10,PRF-52680,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,257261,Released,1
55,257265,Released,9
56,257266,Released,1
57,257268,Released,1
58,257269,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257261,NaN,NaN,1.0
257265,NaN,NaN,9.0
257266,NaN,NaN,1.0
257268,NaN,NaN,1.0
257269,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257186,1.0,1.0,1.0
257194,35.0,5.0,1.0
257195,0.0,38.0,4.0
257198,0.0,1.0,0.0
257199,0.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257186,1,1,1
257194,35,5,1
257195,0,38,4
257198,0,1,0
257199,0,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257186,1,1,1
1,257194,35,5,1
2,257195,0,38,4
3,257198,0,1,0
4,257199,0,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257186,1,1,1
1,257194,35,5,1
2,257195,,38,4
3,257198,,1,
4,257199,,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc."
3,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health
4,Feb 13 2023 12:38PM,257265,10,Bio-PRF
13,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc."
14,Feb 13 2023 12:25PM,257261,10,Bio-PRF
15,Feb 13 2023 12:16PM,257260,10,"Nextsource Biotechnology, LLC"
18,Feb 13 2023 12:11PM,257259,19,"NAPP Technologies, LLC"
19,Feb 13 2023 12:09PM,257258,19,"NAPP Technologies, LLC"
20,Feb 13 2023 12:03PM,257257,10,Yusen – 3D Matrix
25,Feb 13 2023 12:03PM,257256,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc.",,,3
1,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health,,,1
2,Feb 13 2023 12:38PM,257265,10,Bio-PRF,,,9
3,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc.",,,1
4,Feb 13 2023 12:25PM,257261,10,Bio-PRF,,,1
5,Feb 13 2023 12:16PM,257260,10,"Nextsource Biotechnology, LLC",,,3
6,Feb 13 2023 12:11PM,257259,19,"NAPP Technologies, LLC",,,1
7,Feb 13 2023 12:09PM,257258,19,"NAPP Technologies, LLC",,,1
8,Feb 13 2023 12:03PM,257257,10,Yusen – 3D Matrix,,1,4
9,Feb 13 2023 12:03PM,257256,10,ISDIN Corporation,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc.",3,,
1,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health,1,,
2,Feb 13 2023 12:38PM,257265,10,Bio-PRF,9,,
3,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc.",1,,
4,Feb 13 2023 12:25PM,257261,10,Bio-PRF,1,,
5,Feb 13 2023 12:16PM,257260,10,"Nextsource Biotechnology, LLC",3,,
6,Feb 13 2023 12:11PM,257259,19,"NAPP Technologies, LLC",1,,
7,Feb 13 2023 12:09PM,257258,19,"NAPP Technologies, LLC",1,,
8,Feb 13 2023 12:03PM,257257,10,Yusen – 3D Matrix,4,1,
9,Feb 13 2023 12:03PM,257256,10,ISDIN Corporation,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc.",3,,
1,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health,1,,
2,Feb 13 2023 12:38PM,257265,10,Bio-PRF,9,,
3,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc.",1,,
4,Feb 13 2023 12:25PM,257261,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc.",3.0,NaN,NaN
1,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health,1.0,NaN,NaN
2,Feb 13 2023 12:38PM,257265,10,Bio-PRF,9.0,NaN,NaN
3,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Feb 13 2023 12:25PM,257261,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc.",3.0,0.0,0.0
1,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health,1.0,0.0,0.0
2,Feb 13 2023 12:38PM,257265,10,Bio-PRF,9.0,0.0,0.0
3,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Feb 13 2023 12:25PM,257261,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7202499,389.0,2.0,1.0
12,1028921,3.0,3.0,0.0
15,771677,31.0,65.0,7.0
19,2057849,38.0,67.0,35.0
21,771707,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7202499,389.0,2.0,1.0
1,12,1028921,3.0,3.0,0.0
2,15,771677,31.0,65.0,7.0
3,19,2057849,38.0,67.0,35.0
4,21,771707,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,389.0,2.0,1.0
1,12,3.0,3.0,0.0
2,15,31.0,65.0,7.0
3,19,38.0,67.0,35.0
4,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,389.0
1,12,Released,3.0
2,15,Released,31.0
3,19,Released,38.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,1.0,0.0,7.0,35.0,0.0
Executing,2.0,3.0,65.0,67.0,1.0
Released,389.0,3.0,31.0,38.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,1.0,0.0,7.0,35.0,0.0
1,Executing,2.0,3.0,65.0,67.0,1.0
2,Released,389.0,3.0,31.0,38.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,1.0,0.0,7.0,35.0,0.0
1,Executing,2.0,3.0,65.0,67.0,1.0
2,Released,389.0,3.0,31.0,38.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()